# Simulation template.

In this notebook we produce the results of the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963).

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 650 ms, sys: 44.3 ms, total: 695 ms
Wall time: 689 ms


In [2]:
%%time
### Dataset and simulation parameters
csv_path = "/home/linneu/ssh_csvs/ssh1/periodic_100_6561.csv"   
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = "/home/linneu/ssh_simulations/ssh1/periodic_100_6561"
val_split = 0.9  
features_to_use = None 
shuffle_features = False
random_state = 137                    

### Running a simulation
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy"}
allowed_windings = [0, 1]
val_split = 0.9
features_to_use = None
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/linneu/ssh_csvs/ssh1/periodic_180_6561.csv"
model_name = "DecisionTreeClassifier"
simulation_dir = "/home/linneu/ssh_simulations/ssh1/periodic_180_6561"
random_state = 222


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

/home/linneu/anaconda3/envs/ml_top_phases/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1180980 entries, 0 to 1180979
Columns: 188 entries, id to feat179
dtypes: float64(183), int32(3), object(2)
memory usage: 1.6+ GB
CPU times: user 28 s, sys: 1.51 s, total: 29.5 s
Wall time: 30 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.074536,0.074536,...,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.105409,0.105345,...,0.099052,0.097734,0.101326,0.100250,0.103106,0.102278,0.104383,0.103808,0.105152,0.104832
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.105409,0.105152,...,0.080748,0.075825,0.089392,0.085278,0.096296,0.093071,0.101326,0.099052,0.104383,0.103106
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.074536,0.074536,...,-0.074536,0.074536,-0.074536,0.074536,-0.074536,0.074536,-0.074536,0.074536,-0.074536,0.074536
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.000030,-0.003649,...,-0.036024,-0.039459,-0.029026,-0.032545,-0.021886,-0.025472,-0.014640,-0.018274,-0.007323,-0.010988


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  180
n_hamiltonians:  6561
n_ts:  2
CPU times: user 1.74 ms, sys: 97 µs, total: 1.84 ms
Wall time: 1.42 ms


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.9358329522938577
% val:  0.0
% test:  0.06416704770614236
% train + val + test:  1.0


number of train hamiltonians:  6140
number of val hamiltonians:  0
number of test hamiltonians:  421
total number of hamiltonians:  6561


train ids:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 161, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   1%|          | 1/100 [00:40<1:07:31, 40.93s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   2%|▏         | 2/100 [01:21<1:06:46, 40.89s/it]

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 100/100 [1:11:41<00:00, 43.82s/it]

running experiments: 100%|██████████| 100/100 [1:11:41<00:00, 43.02s/it]

CPU times: user 1h 11min 8s, sys: 34.5 s, total: 1h 11min 43s
Wall time: 1h 11min 41s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.074536,0.074536,...,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.105409,0.105345,...,0.099052,0.097734,0.101326,0.100250,0.103106,0.102278,0.104383,0.103808,0.105152,0.104832
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.105409,0.105152,...,0.080748,0.075825,0.089392,0.085278,0.096296,0.093071,0.101326,0.099052,0.104383,0.103106
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.074536,0.074536,...,-0.074536,0.074536,-0.074536,0.074536,-0.074536,0.074536,-0.074536,0.074536,-0.074536,0.074536
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.000030,-0.003649,...,-0.036024,-0.039459,-0.029026,-0.032545,-0.021886,-0.025472,-0.014640,-0.018274,-0.007323,-0.010988
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.000365,0.006989,...,0.067476,0.072960,0.055548,0.061662,0.042540,0.049164,0.028703,0.035709,0.014308,0.021558
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.105377,-0.104528,...,-0.050435,-0.040484,-0.068577,-0.059834,-0.083723,-0.076569,-0.095209,-0.089958,-0.102533,-0.099416
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.002187,-0.013191,...,-0.092361,-0.097165,-0.079781,-0.086545,-0.063714,-0.072143,-0.044863,-0.054587,-0.024050,-0.034646
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.105391,0.104094,...,0.016379,0.001727,0.044446,0.030711,0.069070,0.057316,0.088342,0.079480,0.100770,0.095486
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.000845,0.015506,...,0.103951,0.105371,0.095109,0.100508,0.078897,0.087858,0.056573,0.068401,0.029866,0.043644


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 262 ms, sys: 3.98 ms, total: 266 ms
Wall time: 265 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
670178,3723,H_0.25_1.9_1.00125.dat,0.25,1.90,1.001249,1,1,val,0.000022,-0.050172,...,0.036031,0.015440,0.083050,-0.042162,0.104830,-0.086951,0.094751,-0.105315,0.055877,-0.091673
701326,3896,H_0.4_-1.6_1.00087.dat,0.40,-1.60,1.000868,1,1,train,0.105409,-0.082041,...,0.052804,0.016297,-0.103130,0.066488,0.085210,-0.105275,-0.010904,0.074398,-0.070618,0.005711
168087,933,H_-1.45_0.1_6.99737e-05.dat,-1.45,0.10,0.000070,0,0,val,0.000344,0.006705,...,0.000344,0.006705,-0.100143,0.102119,-0.062236,0.056408,0.061679,-0.067257,0.100356,-0.097975
1097189,6095,H_1.75_-1_0.000394397.dat,1.75,-1.00,0.000394,0,0,val,0.096893,0.104615,...,-0.024052,0.030889,0.101727,0.101799,0.031152,-0.023783,-0.099552,-0.103459,-0.038101,0.016562
201038,1116,H_-1.35_1.15_0.000498913.dat,-1.35,1.15,0.000499,0,0,val,0.105405,-0.054431,...,-0.098735,0.020275,-0.064171,-0.037621,-0.010105,-0.084084,0.047032,-0.104994,0.089875,-0.093995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559257,3106,H_-0.1_-0.6_1.0013.dat,-0.10,-0.60,1.001302,1,1,val,0.105409,-0.071743,...,0.098985,-0.041004,-0.099119,0.093830,0.052874,-0.102752,0.018111,0.063596,-0.080622,0.005318
506768,2815,H_-0.3_1.05_1.00084.dat,-0.30,1.05,1.000844,1,1,val,-0.105409,0.055888,...,0.105409,-0.055888,-0.032573,0.102270,-0.085278,-0.007318,0.085278,-0.097747,0.032573,0.067729
726889,4038,H_0.45_1.45_1.00157.dat,0.45,1.45,1.001573,1,1,val,-0.105407,-0.087733,...,0.081151,0.029650,0.105107,0.091595,0.070064,0.102127,-0.004307,0.055332,-0.076260,-0.022521
8282,46,H_-2_0.3_0.000141469.dat,-2.00,0.30,0.000141,0,0,val,0.105409,0.105392,...,-0.080748,-0.079504,0.089392,0.088362,-0.096296,-0.095501,0.101326,0.100781,-0.104383,-0.104100


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.09358329522938577
% val:  0.8422496570644719
% test:  0.06416704770614236
% train + val + test:  1.0


number of train hamiltonians:  614
number of val hamiltonians:  5526
number of test hamiltonians:  421
total number of hamiltonians:  6561


train ids:  [2247, 3469, 4202, 1347, 5115, 5014, 2631, 3866, 6197, 481, 5603, 3299, 6178, 5788, 1144, 48, 5072, 824, 789, 4305, 1612, 3571, 1281, 291, 4247, 5370, 515, 3130, 3896, 4611, 6163, 5462, 5418, 3781, 1402, 5402, 4711, 764, 2443, 3262, 3132, 2106, 6029, 2614, 16, 3111, 3717, 910, 1239, 3614, 5618, 611, 2695, 3934, 3324, 2584, 2067, 6354, 88, 4540, 732, 1430, 5316, 1233, 2525, 2686, 6325, 5925, 3560, 4068, 1370, 3447, 1672, 784, 2736, 723, 3673, 4481, 950, 6045, 3857, 4222, 5980, 3812, 1793, 4372, 4090, 1810, 5694, 798, 1801, 1271, 6382, 607, 993, 230, 1292, 1765, 4578, 6189, 5800, 3953, 3011, 853, 5314, 4373, 3671, 2479, 5219, 1107, 4770, 4938, 2979, 285, 1270, 6327, 4802, 5080, 1375, 1716, 3825, 4985, 5039, 6512, 33, 1206, 3

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 282 µs, sys: 3 µs, total: 285 µs
Wall time: 173 µs


,t1,t2,type_of,0,1,phase,pred_phase
id,,,,,,,
0,-2.0,-2.00,test,0.588889,0.411111,999,0
1,-2.0,-1.95,test,0.733333,0.266667,0,0
2,-2.0,-1.90,test,0.783333,0.216667,0,0
3,-2.0,-1.85,test,0.894444,0.105556,0,0
4,-2.0,-1.80,val,0.944444,0.055556,0,0
...,...,...,...,...,...,...,...
6556,2.0,1.80,val,0.950000,0.050000,0,0
6557,2.0,1.85,test,0.855556,0.144444,0,0
6558,2.0,1.90,test,0.822222,0.177778,0,0


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  1180980


,id,phase,pred_phase,type_of
0,0,999,1,test
1,0,999,0,test
2,0,999,1,test
3,0,999,0,test
4,0,999,1,test
...,...,...,...,...
1180975,6560,999,0,test
1180976,6560,999,1,test
1180977,6560,999,0,test
1180978,6560,999,0,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.9892870068765834,
 'eigenvector_val': 0.971933687215989,
 'eigenvector_test': 0.7841025641025641,
 'hamiltonian_train': 1.0,
 'hamiltonian_val': 1.0,
 'hamiltonian_test': 1.0}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}